In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import normalize
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
import lightgbm as LGBMRegressor

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd gdrive/My\ Drive/volcano

/content/gdrive/My Drive/volcano


In [ ]:
train = pd.read_csv('train.csv')
train.head()

,segment_id,time_to_eruption
0,1136037770,12262005
1,1969647810,32739612
2,1895879680,14965999
3,2068207140,26469720
4,192955606,31072429


In [ ]:
train = train.rename(columns = {"segment_id":"0"})

In [ ]:
training_data = pd.read_csv('training.csv')

In [ ]:
training_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160
0,800654756,4065.0,-3891.0,4.114265,470.541025,0.210296,5.530825,0.008744,1109.0,356.0,104.0,-104.0,-341.0,-560.0,79858.414240,1.636940e+06,-2.246868e+06,32767.0,-28726.0,3.180480,1045.157443,0.637346,246.202077,0.003043,1719.0,325.0,94.0,-89.0,-317.0,-508.0,181349.631218,1.715857e+06,-2.119405e+06,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,746.0,225.0,-223.0,-753.0,-1193.0,153653.257588,4.334366e+06,-6.144321e+06,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000e+00,10389.0,-10709.0,8.997183,927.938539,-0.219028,11.989392,0.009696,2485.0,612.0,179.0,-175.0,-594.0,-516.0,161517.065584,4.703041e+06,-4.405385e+06
1,321543978,2610.0,-2165.0,-0.145998,584.056268,0.006554,0.266015,-0.000250,1407.0,480.0,142.0,-144.0,-475.0,743.0,101165.787658,2.308403e+06,-2.160397e+06,4790.0,-3842.0,-1.840353,808.543726,-0.021848,0.948116,-0.002276,2004.0,629.0,183.0,-183.0,-633.0,-385.0,140239.896756,1.846995e+06,-1.821633e+06,2127.0,-2484.0,-2.642639,583.095391,-0.018962,0.173094,-0.004532,...,515.0,153.0,-151.0,-508.0,328.0,107210.354523,3.330505e+06,-4.454254e+06,2825.0,-3032.0,3.427993,680.009219,0.002706,0.200390,0.005041,1604.0,564.0,159.0,-161.0,-552.0,494.0,116300.766483,2.185233e+06,-3.534504e+06,4744.0,-4234.0,0.070965,943.018011,0.014998,0.510004,0.000075,2239.0,755.0,227.0,-224.0,-765.0,615.0,166992.383948,2.671369e+06,-3.365313e+06
2,1417547769,1435.0,-1242.0,0.825053,262.681761,0.074320,1.083577,0.003141,680.0,205.0,60.0,-61.0,-204.0,115.0,45889.584750,8.456905e+05,-8.588091e+05,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000e+00,1704.0,-1432.0,2.783204,247.929440,0.122181,1.559794,0.011226,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000e+00,1794.0,-1496.0,0.002567,289.996479,0.012442,1.297353,0.000009,738.0,220.0,64.0,-64.0,-220.0,-192.0,50227.600600,7.037183e+05,-7.858183e+05,3404.0,-3299.0,-3.466092,541.120483,-0.010505,1.433073,-0.006405,1362.0,407.0,120.0,-120.0,-417.0,731.0,92964.322382,1.183025e+06,-1.452741e+06
3,729870090,2370.0,-2310.0,-1.490725,598.160014,-0.009541,-0.181077,-0.002492,1349.0,517.0,161.0,-154.0,-524.0,-192.0,106243.528332,5.147985e+06,-4.225620e+06,8138.0,-10744.0,1.144414,1128.968653,-0.061617,2.513003,0.001014,2812.0,854.0,253.0,-247.0,-850.0,142.0,193965.632299,2.985203e+06,-2.669410e+06,1839.0,-2177.0,-5.370710,526.233915,-0.044986,-0.006676,-0.010206,...,1310.0,362.0,-389.0,-1304.0,262.0,274438.998757,1.384737e+07,-1.440116e+07,2461.0,-2312.0,7.548691,541.282557,0.059947,0.252700,0.013946,1311.0,454.0,137.0,-130.0,-441.0,570.0,93958.454135,3.821611e+06,-2.505777e+06,4738.0,-4398.0,15.602507,1151.480179,0.045531,-0.079678,0.013550,2678.0,1007.0,298.0,-297.0,-966.0,-1765.0,194468.612003,1.020410e+07,-1.032877e+07
4,830695026,3812.0,-2717.0,-0.737071,399.194145,0.351813,8.512710,-0.001846,1044.0,263.0,72.0,-77.0,-269.0,-14.0,70332.989013,1.673152e+06,-1.345567e+06,5578.0,-5526.0,6.750504,586.609285,0.135370,9.582252,0.011508,1620.0,381.0,112.0,-110.0,-374.0,359.0,101211.474952,1.216604e+06,-1.664026e+06,2048.0,-2822.0,3.902768,331.745285,-0.052079,3.931315,0.011764,...,395.0,121.0,-120.0,-399.0,478.0,85355.594108,3.638734e+06,-2.994617e+06,2467.0,-2154.0,1.566774,343.401534,-0.002466,2.108201,0.004563,864.0,260.0,76.0,-73.0,-257.0,366.0,58806.280710,1.145501e+06,-9.496929e+05,4774.0,-4081.0,4.981517,699.031040,0.272209,3.026470,0.007126,1902.0,509.0,148.0,-150.0,-508.0,-456.0,123340.842048,4.113141e+06,-2.808837e+06


In [ ]:
train_df = pd.merge(training_data, train, on='0')

In [ ]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,time_to_eruption
0,800654756,4065.0,-3891.0,4.114265,470.541025,0.210296,5.530825,0.008744,1109.0,356.0,104.0,-104.0,-341.0,-560.0,79858.414240,1.636940e+06,-2.246868e+06,32767.0,-28726.0,3.180480,1045.157443,0.637346,246.202077,0.003043,1719.0,325.0,94.0,-89.0,-317.0,-508.0,181349.631218,1.715857e+06,-2.119405e+06,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,225.0,-223.0,-753.0,-1193.0,153653.257588,4.334366e+06,-6.144321e+06,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000e+00,10389.0,-10709.0,8.997183,927.938539,-0.219028,11.989392,0.009696,2485.0,612.0,179.0,-175.0,-594.0,-516.0,161517.065584,4.703041e+06,-4.405385e+06,16818516
1,321543978,2610.0,-2165.0,-0.145998,584.056268,0.006554,0.266015,-0.000250,1407.0,480.0,142.0,-144.0,-475.0,743.0,101165.787658,2.308403e+06,-2.160397e+06,4790.0,-3842.0,-1.840353,808.543726,-0.021848,0.948116,-0.002276,2004.0,629.0,183.0,-183.0,-633.0,-385.0,140239.896756,1.846995e+06,-1.821633e+06,2127.0,-2484.0,-2.642639,583.095391,-0.018962,0.173094,-0.004532,...,153.0,-151.0,-508.0,328.0,107210.354523,3.330505e+06,-4.454254e+06,2825.0,-3032.0,3.427993,680.009219,0.002706,0.200390,0.005041,1604.0,564.0,159.0,-161.0,-552.0,494.0,116300.766483,2.185233e+06,-3.534504e+06,4744.0,-4234.0,0.070965,943.018011,0.014998,0.510004,0.000075,2239.0,755.0,227.0,-224.0,-765.0,615.0,166992.383948,2.671369e+06,-3.365313e+06,10340827
2,1417547769,1435.0,-1242.0,0.825053,262.681761,0.074320,1.083577,0.003141,680.0,205.0,60.0,-61.0,-204.0,115.0,45889.584750,8.456905e+05,-8.588091e+05,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000e+00,1704.0,-1432.0,2.783204,247.929440,0.122181,1.559794,0.011226,...,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000e+00,1794.0,-1496.0,0.002567,289.996479,0.012442,1.297353,0.000009,738.0,220.0,64.0,-64.0,-220.0,-192.0,50227.600600,7.037183e+05,-7.858183e+05,3404.0,-3299.0,-3.466092,541.120483,-0.010505,1.433073,-0.006405,1362.0,407.0,120.0,-120.0,-417.0,731.0,92964.322382,1.183025e+06,-1.452741e+06,40087733
3,729870090,2370.0,-2310.0,-1.490725,598.160014,-0.009541,-0.181077,-0.002492,1349.0,517.0,161.0,-154.0,-524.0,-192.0,106243.528332,5.147985e+06,-4.225620e+06,8138.0,-10744.0,1.144414,1128.968653,-0.061617,2.513003,0.001014,2812.0,854.0,253.0,-247.0,-850.0,142.0,193965.632299,2.985203e+06,-2.669410e+06,1839.0,-2177.0,-5.370710,526.233915,-0.044986,-0.006676,-0.010206,...,362.0,-389.0,-1304.0,262.0,274438.998757,1.384737e+07,-1.440116e+07,2461.0,-2312.0,7.548691,541.282557,0.059947,0.252700,0.013946,1311.0,454.0,137.0,-130.0,-441.0,570.0,93958.454135,3.821611e+06,-2.505777e+06,4738.0,-4398.0,15.602507,1151.480179,0.045531,-0.079678,0.013550,2678.0,1007.0,298.0,-297.0,-966.0,-1765.0,194468.612003,1.020410e+07,-1.032877e+07,31317486
4,830695026,3812.0,-2717.0,-0.737071,399.194145,0.351813,8.512710,-0.001846,1044.0,263.0,72.0,-77.0,-269.0,-14.0,70332.989013,1.673152e+06,-1.345567e+06,5578.0,-5526.0,6.750504,586.609285,0.135370,9.582252,0.011508,1620.0,381.0,112.0,-110.0,-374.0,359.0,101211.474952,1.216604e+06,-1.664026e+06,2048.0,-2822.0,3.902768,331.745285,-0.052079,3.931315,0.011764,...,121.0,-120.0,-399.0,478.0,85355.594108,3.638734e+06,-2.994617e+06,2467.0,-2154.0,1.566774,343.401534,-0.002466,2.108201,0.004563,864.0,260.0,76.0,-73.0,-257.0,366.0,58806.280710,1.145501e+06,-9.496929e+05,4774.0,-4081.0,4.981517,699.031040,0.272209,3.026470,0.007126,1902.0,509.0,148.0,-150.0,-508.0,-456.0,123340.842048,4.113141e+06,-2.808837e+06,5801334


In [ ]:
data = train_df.values

In [ ]:
data.shape

(4431, 162)

In [ ]:
X = data[:,1:-1]
X = normalize(X)
Y = data[:,-1]

In [ ]:
acf = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(max_depth=20),
                        n_estimators = 150,
                        learning_rate = 0.2)

In [ ]:
acf.fit(X,Y)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(ccp_alpha=0.0,
                                                       criterion='mse',
                                                       max_depth=20,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                                       presort='deprecated',
                                                       random_state=None,
                                                       splitter='best'),
                 

In [ ]:
train_data = lgb.Dataset(X, label=Y)
params = { 'num_leaves': 85,
          'n_estimators': 1000,
    'min_data_in_leaf': 10, 
    'objective':'mae',
    'max_depth': -1,
    'learning_rate': 0.01,
    'max_bins': 2048,
    "boosting": "gbdt",
    "feature_fraction": 0.91,
    "bagging_freq": 1,
    "bagging_fraction": 0.91,
    "bagging_seed": 42,
    "metric": 'mae',
    "lambda_l1": 0.1,
    "verbosity": -1,
    "nthread": -1,
    "random_state": 42}
acf2 = LGBMRegressor.train(params=params, train_set=train_data)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [ ]:
testing = pd.read_csv('testing.csv')
testing = testing.rename(columns = {"0":"segment_id"})
ss = pd.read_csv('sample_submission.csv')
ss_df = ss.drop(columns = ['time_to_eruption'])
test_df = pd.merge(ss_df, testing,on='segment_id')

In [ ]:
test_df.head()

,segment_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160
0,1000213997,1347.0,-1319.0,4.462476,306.171923,0.018571,0.441679,0.014575,758.0,252.0,73.0,-72.0,-244.0,9.405932e+05,-1.193852e+06,180.0,53170.191532,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,1687.0,-1282.0,3.584690,305.584622,0.085503,0.633998,0.011731,...,301.0,90.0,-90.0,-306.0,2.916333e+06,-5.610849e+06,-288.0,70005.956847,1630.0,-1473.0,-2.540874,320.738774,0.006616,0.429972,-0.007922,766.0,255.0,77.0,-77.0,-264.0,1.009912e+06,-928557.783796,152.0,54661.959638,0.0,0.0,0.000000,0.000000,0.000000,-3.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000
1,100023368,1214.0,-1268.0,0.838103,335.588304,-0.011491,-0.097786,0.002497,768.0,289.0,87.0,-84.0,-287.0,2.409332e+06,-1.739455e+06,1012.0,58347.129711,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,1154.0,-1290.0,1.009217,279.857480,-0.055098,0.208645,0.003606,...,430.0,127.0,-133.0,-436.0,3.548241e+06,-2.961625e+06,901.0,87207.248179,1164.0,-1296.0,2.450759,288.561663,-0.032078,0.269738,0.008493,684.0,242.0,72.0,-70.0,-235.0,6.940473e+05,-642676.277747,69.0,49622.129061,2820.0,-2502.0,-5.790420,569.063589,-0.044316,0.173974,-0.010175,1304.0,469.0,139.0,-145.0,-482.0,2.450837e+06,-2.194674e+06,103.0,98043.128081
2,1000488999,4259.0,-3806.0,0.794903,352.169313,-0.050055,6.125070,0.002257,927.0,245.0,72.0,-71.0,-243.0,7.435363e+05,-9.281370e+05,66.0,61106.697207,3140.0,-3552.0,-4.740871,506.428522,-0.079324,1.494615,-0.009361,1245.0,388.0,115.0,-119.0,-396.0,1.018407e+06,-8.384089e+05,375.0,88340.178883,806.0,-637.0,2.116098,190.870729,0.092696,0.043278,0.011087,...,345.0,101.0,-99.0,-336.0,3.193039e+06,-2.169692e+06,665.0,70463.998531,912.0,-776.0,-0.932868,205.033011,0.017395,0.108515,-0.004550,485.0,169.0,50.0,-51.0,-173.0,4.943979e+05,-526157.690390,169.0,35643.458797,0.0,0.0,0.000000,0.000000,0.000000,-3.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000
3,1001028887,1220.0,-1156.0,1.953717,259.027964,0.014705,0.083159,0.007542,609.0,221.0,66.0,-66.0,-216.0,7.332782e+05,-8.965958e+05,-303.0,44865.868107,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,850.0,-1043.0,0.404360,230.132566,-0.027923,0.096697,0.001757,...,437.0,131.0,-133.0,-430.0,3.700346e+06,-3.490077e+06,277.0,99524.230489,1144.0,-1103.0,0.783520,253.729837,0.017976,0.198595,0.003088,608.0,212.0,63.0,-61.0,-212.0,5.516028e+05,-705086.279262,-120.0,44201.230401,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000
4,1001857862,972.0,-1111.0,-3.984584,254.304307,-0.077958,0.058496,-0.015669,577.0,209.0,62.0,-62.0,-215.0,1.108086e+06,-9.995154e+05,118.0,45152.572490,11615.0,-10521.0,-3.306728,746.298843,0.086846,11.154690,-0.004431,1827.0,529.0,156.0,-157.0,-540.0,1.656500e+06,-1.409113e+06,-546.0,128699.202039,1129.0,-1011.0,0.898585,249.386849,0.002132,0.138126,0.003603,...,484.0,145.0,-144.0,-473.0,4.307985e+06,-2.296222e+06,-910.0,103382.824961,1301.0,-1230.0,3.618723,304.521761,0.050748,0.191436,0.011883,736.0,255.0,76.0,-74.0,-248.0,7.041937e+05,-717373.347585,555.0,52571.793407,2265.0,-2638.0,-0.178297,525.525354,0.001451,0.165441,-0.000339,1222.0,434.0,130.0,-130.0,-437.0,1.338345e+06,-1.480067e+06,66.0,91297.115447


In [ ]:
X_test = test_df.values[:,1:]

In [ ]:
X_test.shape

(4520, 160)

In [ ]:
fc = test_df.values[:,0]
fc = fc.reshape((fc.shape[0],1))

In [ ]:
X_test = normalize(X_test)

In [ ]:
preds = acf.predict(X_test)

In [ ]:
preds = preds.reshape((preds.shape[0],1))

In [ ]:
preds2 = acf2.predict(X_test)
preds2 = preds2.reshape((preds2.shape[0],1))

In [ ]:
sub_data2 = np.concatenate((fc, preds2),axis = 1)

In [ ]:
sub_data = np.concatenate((fc,preds),axis = 1)

In [ ]:
sub = pd.DataFrame(sub_data, columns=['segment_id','time_to_eruption'])

In [ ]:
sub.head()

,segment_id,time_to_eruption
0,1.000214e+09,23565849.0
1,1.000234e+08,22763676.0
2,1.000489e+09,27077969.0
3,1.001029e+09,26223811.0
4,1.001858e+09,22499218.0


In [ ]:
sub2 = pd.DataFrame(sub_data2, columns=['segment_id','time_to_eruption'])

In [ ]:
sub['segment_id'] = sub['segment_id'].astype('int32')
sub.to_csv('submission2.csv',index=False)

In [ ]:
sub2['segment_id'] = sub2['segment_id'].astype('int32')
sub2.to_csv('submission3.csv',index=False)

In [ ]:
from google.colab import files
files.download('submission3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>